<a href="https://colab.research.google.com/github/lkhart/kaggle_house_prices/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **House Price Data Prep**

After conducting exploratory data analysis, prep data for modeling through feature engineering. This notebook will apply transformations identified in EDA to both train and test data, and should be run before the modeling notebook(s).



## Setup

In [1]:
import pandas as pd
import numpy as np


In [17]:
# import train set from github
url_train = 'https://raw.githubusercontent.com/lkhart/kaggle_house_prices/main/data/train.csv'

train_df = pd.read_csv(url_train)

print(train_df.shape)
train_df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [18]:
# import test set from github
url_test = 'https://raw.githubusercontent.com/lkhart/kaggle_house_prices/main/data/test.csv'

test_df = pd.read_csv(url_test)

print(test_df.shape)
test_df.head()

(1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


## Train DF Prep

### Handle null values

In [4]:
# replace NA values with 'None' or 0
train_df.PoolQC.fillna('None', inplace=True)
train_df.MiscFeature.fillna('None', inplace=True)
train_df.Alley.fillna('None', inplace=True)
train_df.Fence.fillna('None', inplace=True)
train_df.FireplaceQu.fillna('None', inplace=True)
train_df.GarageCond.fillna('None', inplace=True)
train_df.GarageType.fillna('None', inplace=True)
train_df.GarageYrBlt.fillna('None', inplace=True)
train_df.GarageFinish.fillna('None', inplace=True)
train_df.GarageQual.fillna('None', inplace=True)
train_df.BsmtExposure.fillna('None', inplace=True)
train_df.BsmtFinType2.fillna('None', inplace=True)
train_df.BsmtFinType1.fillna('None', inplace=True)
train_df.BsmtCond.fillna('None', inplace=True)
train_df.BsmtQual.fillna('None', inplace=True)
train_df.MasVnrType.fillna('None', inplace=True)
train_df.MasVnrArea.fillna(0, inplace=True)

# replace Electrical NA w most common value
train_df.Electrical.fillna(train_df.Electrical.mode()[0], inplace=True)

# replace LotFrontage NA w median LotFrontage for associated Neighborhood
train_df.LotFrontage = train_df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# confirm that all NA values have been addressed
# get counts of NA values
ct_na = train_df.isna().sum().sort_values(ascending=False)
ct_na[ct_na > 0]

Series([], dtype: int64)

### Create new features

In [5]:
# YearBuiltBin
# year built as a categorical variable with binned ranges

# function to bin years
def get_year_built_bin(YearBuilt):
  if YearBuilt < 1960:
    return 'pre-1960'

  elif YearBuilt < 1980:
    return '1960-79'

  elif YearBuilt < 2000:
    return '1980-1999'

  elif YearBuilt < 2006:
    return '2000-2005'
  
  else:
    return '2005 or later'


train_df['YearBuiltBin'] = train_df['YearBuilt'].apply(get_year_built_bin)

In [6]:
# TotRmsAbvGr as a categorical variable with binned ranges

# function to bin 
def get_rooms_bin(TotRmsAbvGrd):
  if TotRmsAbvGrd <= 5:
    return '5 or fewer'

  elif TotRmsAbvGrd <= 8:
    return '6-8'
  
  else:
    return '9 or more'


train_df['TotRmsBin'] = train_df['TotRmsAbvGrd'].apply(get_rooms_bin)

In [7]:
# new variable: HasPool
train_df['HasPool'] = np.where(train_df['PoolArea'] == 0, '0', '1')
# new variable: NumFam
train_df['NumFam'] = np.where((train_df['BldgType'] == '2fmCon') | (train_df['BldgType'] == 'Duplex'), 'Multi', 'Single')
# new variable: RelativeCond
train_df['RelativeCond'] = np.where(train_df['OverallCond'] >= 5, 'Above Average', 'Below Average')
# new variable: LastRemod
train_df['LastRemod'] = np.where(train_df['YearRemodAdd'] >= 1980, 'Since1980', 'Pre1980')
# new variable - total full baths
train_df['TotalFullBath'] = train_df['FullBath'] + train_df['BsmtFullBath'] 
# HasWoodDeck
train_df['HasWoodDeck'] = np.where(train_df['WoodDeckSF'] > 0, '1', '0')
# HasOpenPorch
train_df['HasOpenPorch'] = np.where(train_df['OpenPorchSF'] > 0, '1', '0')
# HasEnclosedPorch
train_df['HasEnclosedPorch'] = np.where(train_df['EnclosedPorch'] > 0, '1', '0')
# HasScreenPorch
train_df['HasScreenPorch'] = np.where(train_df['ScreenPorch'] > 0, '1', '0')
# PartialSale
train_df['PartialSale'] = np.where(train_df['SaleCondition'] == 'Partial', '1', '0')

### Drop unused columns/features

In [8]:
train_df = train_df.drop(['YearBuilt'], axis = 1)
train_df = train_df.drop(['MSSubClass'], axis = 1)
train_df = train_df.drop(['LotFrontage'], axis = 1)
train_df = train_df.drop(['LotArea'], axis = 1)
train_df = train_df.drop(['Street'], axis = 1)
train_df = train_df.drop(['Alley'], axis = 1)
train_df = train_df.drop(['LotShape'], axis = 1)
train_df = train_df.drop(['LandContour'], axis = 1)
train_df = train_df.drop(['Utilities'], axis = 1)
train_df = train_df.drop(['LotConfig'], axis = 1)
train_df = train_df.drop(['LandSlope'], axis = 1)
train_df = train_df.drop(['Condition1'], axis = 1)
train_df = train_df.drop(['Condition2'], axis = 1)
train_df = train_df.drop(['BldgType'], axis = 1)
train_df = train_df.drop(['HouseStyle'], axis = 1)
train_df = train_df.drop(['OverallCond'], axis = 1)
train_df = train_df.drop(['YearRemodAdd'], axis = 1)
train_df = train_df.drop(['RoofStyle'], axis = 1)
train_df = train_df.drop(['RoofMatl'], axis = 1)
train_df = train_df.drop(['Exterior1st'], axis = 1)
train_df = train_df.drop(['Exterior2nd'], axis = 1)
train_df = train_df.drop(['MasVnrArea'], axis = 1)
train_df = train_df.drop(['ExterCond'], axis = 1)
train_df = train_df.drop(['BsmtCond'], axis = 1)
train_df = train_df.drop(['BsmtExposure'], axis = 1)
train_df = train_df.drop(['BsmtFinType1'], axis = 1)
train_df = train_df.drop(['BsmtFinSF1'], axis = 1)
train_df = train_df.drop(['BsmtFinType2'], axis = 1)
train_df = train_df.drop(['BsmtFinSF2'], axis = 1)
train_df = train_df.drop(['BsmtUnfSF'], axis = 1)
train_df = train_df.drop(['Heating'], axis = 1)
train_df = train_df.drop(['Electrical'], axis = 1)
train_df = train_df.drop(['2ndFlrSF'], axis = 1)
train_df = train_df.drop(['LowQualFinSF'], axis = 1)
train_df = train_df.drop(['BsmtFullBath'], axis = 1)
train_df = train_df.drop(['BsmtHalfBath'], axis = 1)
train_df = train_df.drop(['FullBath'], axis = 1)
train_df = train_df.drop(['HalfBath'], axis = 1)
train_df = train_df.drop(['BedroomAbvGr'], axis = 1)
train_df = train_df.drop(['KitchenAbvGr'], axis = 1)
train_df = train_df.drop(['TotRmsAbvGrd'], axis = 1)
train_df = train_df.drop(['Functional'], axis = 1)
train_df = train_df.drop(['Fireplaces'], axis = 1)
train_df = train_df.drop(['GarageYrBlt'], axis = 1)
train_df = train_df.drop(['GarageQual'], axis = 1)
train_df = train_df.drop(['GarageCond'], axis = 1)
train_df = train_df.drop(['PavedDrive'], axis = 1)
train_df = train_df.drop(['WoodDeckSF'], axis = 1)
train_df = train_df.drop(['OpenPorchSF'], axis = 1)
train_df = train_df.drop(['EnclosedPorch'], axis = 1)
train_df = train_df.drop(['3SsnPorch'], axis = 1)
train_df = train_df.drop(['ScreenPorch'], axis = 1)
train_df = train_df.drop(['PoolArea'], axis = 1)
train_df = train_df.drop(['PoolQC'], axis = 1)
train_df = train_df.drop(['Fence'], axis = 1)
train_df = train_df.drop(['MiscFeature'], axis = 1)
train_df = train_df.drop(['MiscVal'], axis = 1)
train_df = train_df.drop(['MoSold'], axis = 1)
train_df = train_df.drop(['YrSold'], axis = 1)
train_df = train_df.drop(['SaleType'], axis = 1)
train_df = train_df.drop(['SaleCondition'], axis = 1)


In [16]:
train_df_prepped = train_df

train_df_prepped.head()

,Id,MSZoning,Neighborhood,OverallQual,MasVnrType,ExterQual,Foundation,BsmtQual,TotalBsmtSF,HeatingQC,CentralAir,1stFlrSF,GrLivArea,KitchenQual,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,SalePrice,YearBuiltBin,TotRmsBin,HasPool,NumFam,RelativeCond,LastRemod,TotalFullBath,HasWoodDeck,HasOpenPorch,HasEnclosedPorch,HasScreenPorch,PartialSale
0,1,RL,CollgCr,7,BrkFace,Gd,PConc,Gd,856,Ex,Y,856,1710,Gd,None,Attchd,RFn,2,548,208500,2000-2005,6-8,0,Single,Above Average,Since1980,3,0,1,0,0,0
1,2,RL,Veenker,6,None,TA,CBlock,Gd,1262,Ex,Y,1262,1262,TA,TA,Attchd,RFn,2,460,181500,1960-79,6-8,0,Single,Above Average,Pre1980,2,1,0,0,0,0
2,3,RL,CollgCr,7,BrkFace,Gd,PConc,Gd,920,Ex,Y,920,1786,Gd,TA,Attchd,RFn,2,608,223500,2000-2005,6-8,0,Single,Above Average,Since1980,3,0,1,0,0,0
3,4,RL,Crawfor,7,None,TA,BrkTil,TA,756,Gd,Y,961,1717,Gd,Gd,Detchd,Unf,3,642,140000,pre-1960,6-8,0,Single,Above Average,Pre1980,2,0,1,1,0,0
4,5,RL,NoRidge,8,BrkFace,Gd,PConc,Gd,1145,Ex,Y,1145,2198,Gd,TA,Attchd,RFn,3,836,250000,2000-2005,9 or more,0,Single,Above Average,Since1980,3,1,1,0,0,0


## Test DF Prep

### Handle null values

In [10]:
test_df.PoolQC.fillna('None', inplace=True)
test_df.MiscFeature.fillna('None', inplace=True)
test_df.Alley.fillna('None', inplace=True)
test_df.Fence.fillna('None', inplace=True)
test_df.FireplaceQu.fillna('None', inplace=True)
test_df.GarageCond.fillna('None', inplace=True)
test_df.GarageType.fillna('None', inplace=True)
test_df.GarageYrBlt.fillna('None', inplace=True)
test_df.GarageFinish.fillna('None', inplace=True)
test_df.GarageQual.fillna('None', inplace=True)
test_df.BsmtExposure.fillna('None', inplace=True)
test_df.BsmtFinType2.fillna('None', inplace=True)
test_df.BsmtFinType1.fillna('None', inplace=True)
test_df.BsmtCond.fillna('None', inplace=True)
test_df.BsmtQual.fillna('None', inplace=True)
test_df.MasVnrType.fillna('None', inplace=True)
test_df.GarageArea.fillna(0, inplace=True)
test_df.GarageCars.fillna(0, inplace=True)
test_df.TotalBsmtSF.fillna(0, inplace=True)
test_df.BsmtFullBath.fillna(0, inplace=True)
test_df.KitchenQual.fillna(test_df.KitchenQual.mode()[0], inplace=True)

In [11]:
# confirm that all NA values have been addressed
# get counts of NA values
ct_na = test_df.isna().sum().sort_values(ascending=False)
ct_na[ct_na > 0]

LotFrontage     227
MasVnrArea       15
MSZoning          4
Utilities         2
Functional        2
BsmtFullBath      2
BsmtHalfBath      2
BsmtUnfSF         1
Exterior1st       1
SaleType          1
BsmtFinSF1        1
BsmtFinSF2        1
Exterior2nd       1
GarageCars        1
dtype: int64

### Create new features

In [12]:
# yearbuiltbin
test_df['YearBuiltBin'] = test_df['YearBuilt'].apply(get_year_built_bin)
# new variable: NumFam
test_df['NumFam'] = np.where((test_df['BldgType'] == '2fmCon') | (test_df['BldgType'] == 'Duplex'), 'Multi', 'Single')
# new variable: RelativeCond
test_df['RelativeCond'] = np.where(test_df['OverallCond'] >= 5, 'Above Average', 'Below Average')
# new variable - total full baths
test_df['TotalFullBath'] = test_df['FullBath'] + test_df['BsmtFullBath'] 
# replace TotalFullBath = NA with median TotalFullBath for associated Neighborhood
test_df.TotalFullBath = test_df.groupby('Neighborhood')['TotalFullBath'].transform(lambda x: x.fillna(x.median()))
# new variable - TotRmsBin
test_df['TotRmsBin'] = test_df['TotRmsAbvGrd'].apply(get_rooms_bin)
# new variable: HasPool
test_df['HasPool'] = np.where(test_df['PoolArea'] == 0, '0', '1')
# new variable: LastRemod
test_df['LastRemod'] = np.where(test_df['YearRemodAdd'] >= 1980, 'Since1980', 'Pre1980')
# new variable - total full baths
test_df['TotalFullBath'] = test_df['FullBath'] + test_df['BsmtFullBath'] 
# HasWoodDeck
test_df['HasWoodDeck'] = np.where(test_df['WoodDeckSF'] > 0, '1', '0')
# HasOpenPorch
test_df['HasOpenPorch'] = np.where(test_df['OpenPorchSF'] > 0, '1', '0')
# HasEnclosedPorch
test_df['HasEnclosedPorch'] = np.where(test_df['EnclosedPorch'] > 0, '1', '0')
# HasScreenPorch
test_df['HasScreenPorch'] = np.where(test_df['ScreenPorch'] > 0, '1', '0')
# PartialSale
test_df['PartialSale'] = np.where(test_df['SaleCondition'] == 'Partial', '1', '0')

### Drop unused columns/features

In [13]:
cols_to_keep = ['Id', 'TotalBsmtSF', '1stFlrSF', 'GarageArea', 'GrLivArea', 'MSZoning', 'Neighborhood', 'OverallQual', 'MasVnrType', 'ExterQual', 'Foundation', 'BsmtQual', 'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageCars', 'HasPool', 'YearBuiltBin', 'NumFam', 'RelativeCond', 'LastRemod', 'TotalFullBath', 'TotRmsBin', 'HasWoodDeck', 'HasOpenPorch', 'HasEnclosedPorch', 'HasScreenPorch', 'PartialSale']


In [14]:
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,YearBuiltBin,NumFam,RelativeCond,TotalFullBath,TotRmsBin,HasPool,LastRemod,HasWoodDeck,HasOpenPorch,HasEnclosedPorch,HasScreenPorch,PartialSale
0,1461,20,RH,80.0,11622,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,...,1,TA,5,Typ,0,None,Attchd,1961,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,None,MnPrv,None,0,6,2010,WD,Normal,1960-79,Single,Above Average,1.0,5 or fewer,0,Pre1980,1,0,0,1,0
1,1462,20,RL,81.0,14267,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,...,1,Gd,6,Typ,0,None,Attchd,1958,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,None,None,Gar2,12500,6,2010,WD,Normal,pre-1960,Single,Above Average,1.0,6-8,0,Pre1980,1,1,0,0,0
2,1463,60,RL,74.0,13830,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,...,1,TA,6,Typ,1,TA,Attchd,1997,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,None,MnPrv,None,0,3,2010,WD,Normal,1980-1999,Single,Above Average,2.0,6-8,0,Since1980,1,1,0,0,0
3,1464,60,RL,78.0,9978,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,...,1,Gd,7,Typ,1,Gd,Attchd,1998,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,None,None,None,0,6,2010,WD,Normal,1980-1999,Single,Above Average,2.0,6-8,0,Since1980,1,1,0,0,0
4,1465,120,RL,43.0,5005,Pave,None,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,...,1,Gd,5,Typ,0,None,Attchd,1992,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,None,None,None,0,1,2010,WD,Normal,1980-1999,Single,Above Average,2.0,5 or fewer,0,Since1980,0,1,0,1,0


In [15]:
test_df_prepped = test_df[cols_to_keep]